# Tutorial

In [1]:
%matplotlib inline

In [2]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        # super(NeuralNetwork, self).__init__()  # Initializes internal Module state. (Python 2 & 3)
        super().__init__()  # Python 3 only
        self.flatten = nn.Flatten()  # change Size([batch, dim1, dim2, ..., dimN]) into Size([batch, dim1 * dim2 * ... * dimN])
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(dim=1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([2], device='cuda:0')


In [7]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
layer1 = nn.Linear(in_features=28 * 28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0240, -0.0129, -0.3088,  0.3150,  0.3681, -0.5604, -0.1308,  0.3970,
          0.1100, -0.0405,  0.5774,  0.2673, -0.3496,  0.3433,  0.3891, -0.0158,
          0.8871, -0.2001,  0.0502,  0.5852],
        [ 0.3855,  0.1676, -0.1491,  0.5962,  0.1382, -0.1839, -0.1654,  0.6320,
         -0.0593,  0.0346,  0.4010,  0.0664,  0.0635,  0.4181,  0.7895,  0.3732,
          0.3736, -0.0037, -0.2997,  0.2187],
        [ 0.3433,  0.2812, -0.3080,  0.6776,  0.0482, -0.3910, -0.4639,  0.3982,
         -0.0720, -0.1216,  0.2265,  0.0913,  0.0840,  0.5901,  0.2960, -0.1085,
          0.6149, -0.1477,  0.2292,  0.0775]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.3150, 0.3681, 0.0000, 0.0000, 0.3970, 0.1100,
         0.0000, 0.5774, 0.2673, 0.0000, 0.3433, 0.3891, 0.0000, 0.8871, 0.0000,
         0.0502, 0.5852],
        [0.3855, 0.1676, 0.0000, 0.5962, 0.1382, 0.0000, 0.0000, 0.6320, 0.0000,
         0.0346, 0.4010, 0.0664, 0.0635, 0.4181, 0.789

In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)
print(logits)

tensor([[ 0.1119,  0.1697,  0.1613,  0.4375,  0.2416,  0.1671,  0.2358, -0.4817,
         -0.0501,  0.3790],
        [ 0.0450,  0.3229,  0.0495,  0.2361,  0.3127,  0.1710,  0.2387, -0.3779,
          0.0183,  0.2382],
        [ 0.0588,  0.1516,  0.1192,  0.3696,  0.1063,  0.1670,  0.2743, -0.3776,
         -0.0660,  0.3014]], grad_fn=<AddmmBackward>)


In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.0950, 0.1006, 0.0998, 0.1315, 0.1081, 0.1003, 0.1075, 0.0524, 0.0808,
         0.1240],
        [0.0906, 0.1197, 0.0911, 0.1097, 0.1185, 0.1028, 0.1100, 0.0594, 0.0883,
         0.1100],
        [0.0932, 0.1023, 0.0990, 0.1272, 0.0977, 0.1038, 0.1156, 0.0602, 0.0823,
         0.1188]], grad_fn=<SoftmaxBackward>)


In [18]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values: {param[:2]} \n")  # only print part of the params

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[ 0.0122,  0.0301, -0.0039,  ..., -0.0023,  0.0072, -0.0278],
        [-0.0194, -0.0269, -0.0121,  ..., -0.0054,  0.0177, -0.0053]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([ 0.0355, -0.0102], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 2.1920e-02, -2.0422e-02,  1.9535e-05,  ...,  2.5925e-02,
         -4.0062e-02, -1.9650e-02],
        [-3.9202e-02, -3.1425e-02, -3.8142e-02,  ...,  3.152